In [1]:
!pip install -q mediapipe
!pip install -q pyautogui


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!curl -o hand_landmarker.task https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7635k  100 7635k    0     0  1897k      0  0:00:04  0:00:03  0:00:01 1897k0     0  1951k      0  0:00:03  0:00:03 --:--:-- 1951k


In [3]:
import cv2
import mediapipe as mp
import pyautogui
import time
import util

In [4]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (0, 5), (5, 6), (6, 7), (7, 8),
    (5, 9), (9, 10), (10, 11), (11, 12),
    (9, 13), (13, 14), (14, 15), (15, 16),
    (13, 17), (17, 18), (18, 19), (19, 20),
    (0, 17)
]

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='../hand_landmarker.task'),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=1)

detector = HandLandmarker.create_from_options(options)

print("✅ Success! The detector loaded correctly.")

✅ Success! The detector loaded correctly.


I0000 00:00:1769016245.590582  830314 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1769016245.599336  830317 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1769016245.605034  830322 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:
def move_mouse(points, frame_shape):
    """
    Moves the mouse cursor based on the position of the Index Finger Tip (8).
    points: List of (x,y) tuples representing the hand landmarks.
    """
    if len(points) >= 9:
        screen_w, screen_h = pyautogui.size()
        frame_h, frame_w = frame_shape
        index_finger_tip = points[8]

        x = int(index_finger_tip[0] / frame_w * screen_w)
        y = int(index_finger_tip[1] / frame_h * screen_h)

        pyautogui.moveTo(x, y)

In [6]:
def detect_gestures(frame, points):
    """
    Detects if the Thumb and Index finger are pinching.
    points: List of (x,y) tuples representing the hand landmarks.
    """
    if len(points) >= 21:
        thumb_tip = points[4]
        index_tip = points[8]

        cv2.line(frame, thumb_tip, index_tip, (255, 0, 0), 2)

        dist = util.get_distance([thumb_tip, index_tip])

        if dist < 50 and util.get_angle(points[5], points[6], points[8]) > 90:
            cv2.circle(frame, index_tip, 10, (0, 255, 0), -1)
            cv2.putText(frame, "PINCH / CLICK", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            h, w, _ = frame.shape
            move_mouse(points, (h, w))
        else:
            cv2.putText(frame, f"Dist: {int(dist)}", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

In [7]:
def main():
    cap = cv2.VideoCapture(0)

    is_running = True

    try:
        while is_running and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frameRGB)

            frame_timestamp_ms = int(time.time() * 1000)

            detection_result = detector.detect_for_video(mp_image, frame_timestamp_ms)

            if detection_result.hand_landmarks:
                for hand_landmarks in detection_result.hand_landmarks:

                    h, w, c = frame.shape
                    points = []
                    for lm in hand_landmarks:
                        cx, cy = int(lm.x * w), int(lm.y * h)
                        points.append((cx, cy))

                    for connection in HAND_CONNECTIONS:
                        start_idx = connection[0]
                        end_idx = connection[1]
                        cv2.line(frame, points[start_idx], points[end_idx], (0, 255, 0), 2)

                    for (cx, cy) in points:
                        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

                    detect_gestures(frame, points)

            cv2.imshow('Webcam', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("Quit detected via Keyboard")
                is_running = False
            if cv2.getWindowProperty('Webcam', cv2.WND_PROP_VISIBLE) < 1:
                print("Quit detected via Window Close")
                is_running = False
    finally:
        print("Cleaning up resources...")
        cap.release()
        cv2.destroyAllWindows()
        detector.close()
        for i in range(5):
            cv2.waitKey(1)
        print("Done.")

In [8]:
if __name__ == "__main__":
  main()

W0000 00:00:1769016246.974394  830322 landmark_projection_calculator.cc:81] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Quit detected via Keyboard
Cleaning up resources...
Done.
